In [8]:
%matplotlib inline
cur_dir = '/home/rob/Udacity/capstone/data'
import os
from collections import defaultdict
import cv2
import csv
import numpy as np
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
import sys
csv.field_size_limit(sys.maxsize)

def mask_to_polygons(mask, epsilon=10., min_area=10.):
    # first, find contours with cv2: it's much faster than shapely
    image, contours, hierarchy = cv2.findContours(
        ((mask == 1) * 255).astype(np.uint8),
        cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
    # create approximate contours to have reasonable submission size
    approx_contours = [cv2.approxPolyDP(cnt, epsilon, True)
                       for cnt in contours]
    if not contours:
        return MultiPolygon()
    # now messy stuff to associate parent and child contours
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    # http://docs.opencv.org/3.1.0/d9/d8b/tutorial_py_contours_hierarchy.html
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(approx_contours[idx])
    # create actual polygons filtering by area (removes artifacts)
    all_polygons = []
    for idx, cnt in enumerate(approx_contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    # approximating polygons might have created invalid ones, fix them
    all_polygons = MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        try:
            all_polygons = all_polygons.buffer(0)
        except ValueError:
            print "standard buffer not possible. Use -1"
            try:
                all_polygons = all_polygons.buffer(0)
            except ValueError:
                print "-1 buffer not possible. Use -2"
                all_polygons = all_polygons.buffer(-2)
        # Sometimes buffer() converts a simple Multipolygon to just a Polygon,
        # need to keep it a Multi throughout
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    return all_polygons

def get_scalers(im_size):
    h, w = im_size # they are flipped so that mask_for_polygons works correctly
    h, w = float(h), float(w)
    w_ = w * (w / (w + 1))
    h_ = h * (h / (h + 1))
    return w_ / x_max, h_ / y_min

    
print "all done"

all done


In [45]:
print "started"
trainIM_IDs = []
cur_dir = '/home/rob/Udacity/capstone/data'
with open(cur_dir + '/train_wkt_v4.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar=',')
    for i,row in enumerate(reader):
        if i == 0:
            i = 1
        if (i%10) == 0:
            trainIM_IDs.append(row[0])        

            
""" 
Let's find those optimal parameters: 
for poly in Polys:
for treshold in tresholds
for epsilon in epsilons
for min_area in min_areas
for image in images
    build average jaccard over all images for polys, 
    build average jaccard over all images once for pred masks
    (You will need the train masks!)
"""

direct = 'prediction_masks_16band_trainset/'
tabular = []
counter = 0

tresholds = [.15,.2,.25,.3,.35,.4,.45] #tresholds = [.05 , .15, .30, .40]
epsilons = [0.5,1.,3.]
min_areas = [1.,3.,5.,10.,15.]

errors = np.zeros((10,len(tresholds),len(epsilons),len(min_areas)))
data = np.empty((10,len(tresholds),len(epsilons),len(min_areas)), dtype = np.float16)
for i in range(1,11):
    POLY_TYPE = str(i)
    index = i-1
    print POLY_TYPE
    
    for tres in range(len(tresholds)):
        threshold = tresholds[tres]
        print "tresh:",threshold
        
        for eps in range(len(epsilons)):
            epsilon = epsilons[eps]
            print "eps:", epsilon
            
            for area in range(len(min_areas)):
                min_area = min_areas[area]
                print "minarea:", min_area
                average_jacc = 0
                
                for IM_ID in trainIM_IDs:
                    
                    filename = "pred{}.npz".format(IM_ID)
                    temp = np.load(direct+filename)
                    masks = temp['arr_0'][0]
                    x_max = y_min = None
                    for _im_id, _x, _y in csv.reader(open(cur_dir + '/grid_sizes.csv')):
                        if _im_id == IM_ID:
                            x_max, y_min = float(_x), float(_y)
                            break
                            
                    mask = masks[:,:,index]
                    im_size = mask.shape[:2]  
                    x_scaler, y_scaler = get_scalers(im_size)

                    pred_binary_mask = mask >= threshold

                    pred_polygons = mask_to_polygons(pred_binary_mask, epsilon, min_area)
                    
                    scaled_pred_polygons = shapely.affinity.scale(
                        pred_polygons, xfact=1 / x_scaler, yfact=1 / y_scaler, origin=(0, 0, 0))
                    dumped_prediction = shapely.wkt.dumps(scaled_pred_polygons)
                    final_polygons = shapely.wkt.loads(dumped_prediction)
                    
                    train_polygons = None
                    for _im_id, _poly_type, _poly in csv.reader(open(cur_dir + '/train_wkt_v4.csv')):
                        if _im_id == IM_ID and _poly_type == POLY_TYPE:
                            train_polygons = shapely.wkt.loads(_poly)
                            break
                    try:
                        union = final_polygons.union(train_polygons).area
                    except:
                        print "topological error. Do not use this setup"
                        errors[index, tres, eps, area] = 1
                        final_jacc = 0
                        break
                    if union == 0:
                        final_jacc = 0
                    else:
                        final_jacc = (final_polygons.intersection(train_polygons).area /union  ) 
                        
                    average_jacc += final_jacc
                print "tresh:",threshold, "eps:", epsilon,  "minarea:", min_area 
                average_jacc = average_jacc/25.
                print('Final jaccard', average_jacc)
                data[index,tres,eps,area] = average_jacc

for poly in range(10):
    print np.amax(data[poly,:,:,:])
    print np.argmax(data[poly,:,:,:])


print "job done"

# class 2 very problematic in making polygons for many setups

started
1
tresh: 0.15
eps: 0.5
minarea: 1.0


Self-intersection at or near point 217.66666666666666 562.66666666666663
Self-intersection at or near point 499.44444444444446 177.66666666666666
Self-intersection at or near point 786 66
Self-intersection at or near point 279.75 186.25
Self-intersection at or near point 459.07692307692309 371.76923076923077
Self-intersection at or near point 355 0
Self-intersection at or near point 100.47368421052632 679.63157894736844
Self-intersection at or near point 185 546.5
Self-intersection at or near point 293 190
Self-intersection at or near point 217.66666666666666 562.66666666666663


tresh: 0.15 eps: 0.5 minarea: 1.0
('Final jaccard', 0.07757890627114977)
minarea: 3.0


Self-intersection at or near point 499.44444444444446 177.66666666666666
Self-intersection at or near point 786 66
Self-intersection at or near point 279.75 186.25
Self-intersection at or near point 459.07692307692309 371.76923076923077
Ring Self-intersection at or near point 485 60
Self-intersection at or near point 100.47368421052632 679.63157894736844
Self-intersection at or near point 185 546.5
Self-intersection at or near point 293 190
Self-intersection at or near point 217.66666666666666 562.66666666666663


tresh: 0.15 eps: 0.5 minarea: 3.0
('Final jaccard', 0.07757287572452828)
minarea: 10.0


...

tresh: 0.45 eps: 1.0 minarea: 1.0
('Final jaccard', 0.10968906109723424)
minarea: 3.0


Self-intersection at or near point 100 518


tresh: 0.45 eps: 1.0 minarea: 3.0
('Final jaccard', 0.10986036292933216)
minarea: 10.0


Self-intersection at or near point 100 518


tresh: 0.45 eps: 1.0 minarea: 10.0
('Final jaccard', 0.11149145012690083)
minarea: 15.0
tresh: 0.45 eps: 1.0 minarea: 15.0
('Final jaccard', 0.11241813660092252)
eps: 3.0
minarea: 1.0
tresh: 0.45 eps: 3.0 minarea: 1.0
('Final jaccard', 0.0924588208560402)
minarea: 3.0
tresh: 0.45 eps: 3.0 minarea: 3.0
('Final jaccard', 0.0924588208560402)
minarea: 10.0
tresh: 0.45 eps: 3.0 minarea: 10.0
('Final jaccard', 0.0924588208560402)
minarea: 15.0
tresh: 0.45 eps: 3.0 minarea: 15.0
('Final jaccard', 0.09220619778865739)
9
tresh: 0.15
eps: 0.5
minarea: 1.0
tresh: 0.15 eps: 0.5 minarea: 1.0
('Final jaccard', 0.0)
minarea: 3.0
tresh: 0.15 eps: 0.5 minarea: 3.0
('Final jaccard', 0.0)
minarea: 10.0
tresh: 0.15 eps: 0.5 minarea: 10.0
('Final jaccard', 0.0)
minarea: 15.0
tresh: 0.15 eps: 0.5 minarea: 15.0
('Final jaccard', 0.0)
eps: 1.0
minarea: 1.0
tresh: 0.15 eps: 1.0 minarea: 1.0
('Final jaccard', 0.0)
minarea: 3.0
tresh: 0.15 eps: 1.0 minarea: 3.0
('Final jaccard', 0.0)
minarea: 10.0
tresh: 0.15 ep

In [49]:
print data.shape
print tresholds, epsilons, min_areas
print data[0].shape
for poly in range(10):
    print "Poly:", poly
    print "Order: thresh, eps, area"
    print np.amax(data[poly])
    print np.argmax(data[poly])#
    print len(tresholds), len(epsilons), len(min_areas)
    print data[poly]
    print "Next Poly"

(10, 7, 3, 4)
[0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45] [0.5, 1.0, 3.0] [1.0, 3.0, 10.0, 15.0]
(7, 3, 4)
Poly: 0
Order: thresh, eps, area
0.077942
72
7 3 4
[[[ 0.07757568  0.07757568  0.07739258  0.07720947]
  [ 0.07745361  0.07745361  0.07727051  0.0770874 ]
  [ 0.0760498   0.0760498   0.07598877  0.07592773]]

 [[ 0.07757568  0.07757568  0.07739258  0.07714844]
  [ 0.07751465  0.07751465  0.07733154  0.07714844]
  [ 0.07598877  0.07598877  0.07598877  0.0758667 ]]

 [[ 0.07763672  0.07763672  0.07751465  0.07720947]
  [ 0.07757568  0.07757568  0.07745361  0.07720947]
  [ 0.07611084  0.07611084  0.07611084  0.0760498 ]]

 [[ 0.07769775  0.07769775  0.07757568  0.07733154]
  [ 0.07757568  0.07757568  0.07745361  0.07727051]
  [ 0.07617188  0.07617188  0.07611084  0.0760498 ]]

 [[ 0.07781982  0.07781982  0.07769775  0.07745361]
  [ 0.07751465  0.07757568  0.07745361  0.07720947]
  [ 0.07623291  0.07623291  0.07623291  0.07611084]]

 [[ 0.07788086  0.07788086  0.07775879  0.07751465]
  [ 

In [ ]:
#0 Build: tresh .25, eps0.1, area 3.
#1 misc man : .2, eps5., area 3.
#2 street:  .35, eps1., area 3.
#3 track: .25, 1., 3.
#4 trees .25, 1., 3.
#5 crops .25, 3., 3.
#6 waterway .25, 1., 15.
#7 standing water .35, 1.,3.
#8 cars: doesnt matter. low tresh probably
#9 large cars: same